In [ ]:
import joblib
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from azureml.core import Workspace, Experiment, Run

In [ ]:
ws = Workspace.from_config(path="config.json")

In [ ]:
# Load dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
dataset = read_csv(url, names=names)

# Split-out validation dataset
array = dataset.values
X = array[:,0:4]
y = array[:,4]
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=1)

# Make predictions on validation dataset
model = SVC(gamma='auto')
model.fit(X_train, Y_train)
predictions = model.predict(X_test)

# Evaluate predictions
print(accuracy_score(Y_test, predictions))
print(confusion_matrix(Y_test, predictions))
print(classification_report(Y_test, predictions))

In [ ]:
# save the model to disk
filename = 'models/finalized_iris_model_new.sav'
joblib.dump(model, open(filename, 'wb'))

from azureml.core.model import Model

# Register model
model = Model.register(workspace = ws,
                        model_path=filename,
                        model_name = "iris_model_new",
                        tags = {"ziris": "demo"},
                        description = "Iris Model Zoo",)

print(model.name, model.id, model.version, sep='\t')


In [ ]:
X_test = [[5.7, 3.9, 1.1, 0.1]]
Y_test = ['Iris-setosa']

# load the model from disk
loaded_model = joblib.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

loaded_model.score(X_test, Y_test)

---

In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, AksWebservice, LocalWebservice

# set environment name
env_name = "envxyz"
env = Environment.get(ws, "AzureML-Minimal").clone(env_name)

for pip_package in ["scikit-learn",]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

service_name = 'tirisservice'
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,)

service = Model.deploy(workspace=ws,
                      name=service_name,
                      models=[model],
                      inference_config=inference_config,
                      deployment_config=aci_config,
                      overwrite=True)

service.wait_for_deployment(show_output=True)


---

### Scoring!!

In [ ]:
print(service.scoring_uri)
print(service.swagger_uri)

In [ ]:
import requests
import json

X_test = [[5.7, 3.9, 1.1, 0.1]]
Y_test = ['Iris-setosa']

# URL for the web service
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
key = None

# Two sets of data to score, so we get two results back
data = {
        "data": X_test
       }

# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)


---